<a href="https://colab.research.google.com/github/gideon1971/CMS-convexity/blob/Convexity/Copy_of_Copy_of_SwaptionWeights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examine the sensitivity of CMS convexity calculations based on different initial parameter assumptions #

<h4> acknowledgement to FinancePy for the displaced SABR volatility code https://github.com/domokane/FinancePy </h4>


### Introduction ###

Constant Maturity Swaps use the same framework as vanilla swaps, however, the tenor of the floating index is not in line with the tenor of the floating payments. 

For example, receiver EURIBOR swap will pay an amount at the end of a 6 month coupon per4iod (in arrears). The amount will be based on the EURIBOR 6 month fixing SET IN ADVANCE of a semi-annual floating payment coupon period. \Similarly for OIS swqpw. the floating rate will use the daily compounded rat3s over trhe coupon period.

The floating index used for CMS can be a swap rate of any tenor. 
Pay 20y swap rate

10-year EUR CMS Swap Rate and the 20-year EUR
CMS Swap Rate (displayed on the Reuters page "ICESWAP2").



This would be the rate used to determinbe if a swaption exerfc ises withe a tenor   but the index that fixes is not a term rate or 



### Calculating the weights for for a portfolio of Cash Settled Swaptions to replicate a CMS rate

### Setup
-  Choose the number of swaptions for each Call & Put portfilio. e.g. use 30
-  determine which strikes to use
-  create functions to build the list of strikes & forwards
-  crate a PVBP function



In [7]:
!pip install financepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import pandas as pd
import scipy.stats as si
import matplotlib.pyplot as plt

from financepy.utils import *
from financepy.models.sabr_shifted import *

In [9]:
def N(x):
    Ndist = si.norm.cdf(x,0.0,1.0)
    return Ndist

In [43]:

def B76(ff,kk,tt,vol,ss,cp):# ff= forward, kk = strike, ss=displacement
    
    # create UNDISCOUNTED displaced forwards & srtikes
    ff=ff+ss
    kk=kk+ss

    d1=(np.log(ff/kk) + 0.5* tt * vol**2)/(vol * np.sqrt(tt))
    d2=(np.log(ff/kk) - 0.5* tt * vol**2)/(vol * np.sqrt(tt))

    c=ff*N(d1)-kk*N(d2)
    p=kk*N(-d2)-ff*N(-d1)

    if cp==1:
        retval=c
    else:
        retval=p

    return retval

In [44]:
def getStrikes(anchorStrike, displacement=0, CallPut = "CALL", nCashSettledSwaptions=30):

    if CallPut == "CALL":
        finalStrike = 10 * (anchorStrike + displacement)
    else:
        finalStrike = -displacement + 0.000001

    increment = (finalStrike-anchorStrike)/(nCashSettledSwaptions-1)
    cashSettledSwaptionStrikes = [anchorStrike + i * increment for i in range(nCashSettledSwaptions)]
            
    return cashSettledSwaptionStrikes

In [45]:
def getForwards(anchorStrike, displacement=0, CallPut = "CALL", nCashSettledSwaptions=30):
    
    K=getStrikes(anchorStrike, displacement, CallPut, nCashSettledSwaptions)
    
    increment=K[1]-K[0]

    cashSettledSwaptionForwards =[K[i] + increment/2  for i in range(nCashSettledSwaptions)]
            
    return cashSettledSwaptionForwards

In [46]:
def PVBP(swapParRate,swapTenorInMonths,swapFixedLegPaymentTenorInMonths):
    #calculate the cash settled swaption annuity at exercise

    factor=swapFixedLegPaymentTenorInMonths/12
    nPaymemnts=int(swapTenorInMonths/swapFixedLegPaymentTenorInMonths)
    df = 1/(1 + swapParRate*factor)
            
    s=0.0
    p=1.0
    for i in range(nPaymemnts):
        p*=df
        s+=p
                
    return s * factor

In [47]:
def largest(a,b):
    if a>b:
        r=a
    else:
        r=b
    return r

In [48]:
#The weights can be calculated recursively by calculating theterninal payoff

def calculateWeights(callputN,anchorStrike,displacement,swapTenorinMonths,swapFixedLegMonths,nCashSettledSwaptions=30):
    
    
    CSSweights=[] # create the empty list to hold the weights
    swapTenor=swapTenorinMonths

    if callputN==1:
        CallPut="CALL"
    else:
        CallPut="PUT"
    
    
    Strikes=getStrikes(anchorStrike, displacement, CallPut, nCashSettledSwaptions)
    Forwards=getForwards(anchorStrike, displacement, CallPut, nCashSettledSwaptions)

    # the target rate is the CMS rate at each forward rate
    Targets=[Forwards[i]-anchorStrike for i in range(nCashSettledSwaptions)] 


    for i in range(nCashSettledSwaptions):
        #find weight

        currentFwd=Forwards[i]
        currentPVBP=PVBP(currentFwd,swapTenor,swapFixedLegMonths)

        nFixedStrikes=len(CSSweights) # loopp through the strikes that have already had their weight set
        #print(nFixedStrikes)
        prevPV=0

        if nFixedStrikes>0: # use this once we have calculated the first swaption weight
             for w in range(nFixedStrikes):
                prevPV=prevPV + largest(callputN*(currentFwd-Strikes[w]),0) * currentPVBP * CSSweights[w]
            
        else:
            prevPV=0
           
        requiredPV=Targets[i]-prevPV
        currentWeight= requiredPV / (largest(callputN*(currentFwd-Strikes[i]),0)*currentPVBP)
        CSSweights.append(currentWeight)

    return CSSweights,Strikes

In [70]:
def getConvexityAdjustedSwapRate(F,D,T,swapLengthinMonths,nCashSettledSwaptions,fixedlegfreqmonths, SABRparams):
    
    #Calls
    wCalls, kCalls = calculateWeights(1,F,D,swapLengthinMonths,fixedlegfreqmonths,nCashSettledSwaptions)
    wPuts, kPuts = calculateWeights(-1,F,D,swapLengthinMonths,fixedlegfreqmonths,nCashSettledSwaptions)


    #define shifted SABR parameters
    alpha = SABRparams[0]; beta = SABRparams[1]; rho = SABRparams[2]; nu = SABRparams[3]; shift = D
    #model = SABRShifted(alpha, beta, rho, nu, D)

    #volsSABRCalls = model.black_vol(F, np.array(kCalls),T)
    #volsSABRPuts = model.black_vol(F, np.array(kPuts),T)

    #use another SABR

    volsSABRCalls=[SABR_BlackVol(F+D,k+D,T,alpha, beta, rho, nu) for k in kCalls]
    volsSABRPuts=[SABR_BlackVol(F+D,k+D,T,alpha, beta, rho, nu) for k in kPuts]


    #print(volsSABRCalls)
    #dcefine the annuity of the swap
    ann=PVBP(F,swapLengthinMonths,fixedlegfreqmonths)
    
    PVportfolioPuts=0
    PVportfolioCalls=0

    for s in range(nCashSettledSwaptions):
        kk=kCalls[s]
        w=wCalls[s]
        v=volsSABRCalls[s]

        PVportfolioCalls+=w*ann*B76(F,kk,T,v,D,1)

    for s in range(nCashSettledSwaptions):
        kk=kPuts[s]
        w=wPuts[s]
        v=volsSABRPuts[s]

        PVportfolioPuts+=w*ann*B76(F,kk,T,v,D,-1)
        
        Zcb=np.exp(-F*T)
        return 1*(PVportfolioCalls+PVportfolioPuts)

In [71]:
F=0.03
D=0.00
T=5.0
swapLengthinMonths=240
nCashSettledSwaptions=30

pp=[0.3,1.0,0.5, 0.1]


zzz=getConvexityAdjustedSwapRate(F,D,T, swapLengthinMonths, nCashSettledSwaptions, 6,pp)
print(zzz)
print(F+zzz)

0.0056565573088037495
0.03565655730880375


In [28]:
import numpy as np

def Chi(z , rho):
  xi = np.log((np.sqrt(1 - 2 * rho * z + z ^ 2) + z - rho) / (1 - rho))
  return xi

eps = 0.00000001

def SABR_BlackVol(fwd, k, tau, atm, beta, rho, nu):
    
    # fwd = forward price
    # k = strike price
    # tau = expiry
    # a0 = initial alpha
    # bet = beta
    # rho = rho
    # nu = nu
    a0=AlphaInitial(fwd, tau, atm, beta, rho, nu)
    h = 1 - beta
    p = ( fwd * k )  **  ( h / 2 )
    q = np.log(fwd / k)
    v = h ** 2 * a0 ** 2 /  ( 24 * p ** 2 )  + rho * beta * nu * a0 /  ( 4 * p )  +  ( 2 - 3 * rho ** 2 )  * nu ** 2 / 24
    #note that ((fwd ^ h - k ^ h) / (h * q)) = p * (1 + (h * q) ^ 2 / 24 + (h * q) ^ 4 / 1920 + ...)
    #note that p * q = ((fwd ^ h - k ^ h) / h) / (1 + (h * q) ^ 2 / 24 + (h * q) ^ 4 / 1920 + ...)
    if np.abs(q) < eps:
        eta = p
        zeta_chi = 1
    elif np.abs(h) < eps:
        eta = 1
        zeta = nu / a0 * q
        zeta_chi = zeta / Chi(zeta, rho)
    else:
        eta = ( fwd ** h - k ** h )  /  ( h * q )
        zeta = nu / a0 * p * q
        zeta_chi = zeta / Chi(zeta, rho)

    fn_return_value = zeta_chi *  ( a0 *  ( 1 + v * tau ) )  / eta
    return fn_return_value

def Chi(z, rho):
    fn_return_value = np.log(( np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho )  /  ( 1 - rho ))
    return fn_return_value

def AlphaInitial(fwd, tau, atm, beta, rho, nu):
 
    # fwd = forward
    # tau = expiry
    # atm = at-the-money volatility
    # bet = beta
    # rho = rho
    # nu = nu
    h = 1 - beta
    a = h ** 2 * tau / 24 / fwd **  ( 2 * h )
    b = rho * beta * nu * tau / 4 / fwd ** h
    c = 1 +  ( 2 - 3 * rho ** 2 )  * nu ** 2 * tau / 24
    d = - atm * fwd ** h
    # a*x^3 + b*x^2 + c*x + d = 0
    # take the smallest positive root.
    # When there are three real roots, they are of the order of -1000, 1 and +1000.
    # So we take the root of order 1.
    param_norm = norm(a, b, c, d)
    if np.abs(norm(0, 0, c, d) / param_norm - 1) < eps:
        fn_return_value = - d / c
    elif np.abs(norm(0, b, c, d) / param_norm - 1) < eps:
        fn_return_value = QuadraticSolver(b, c, d)
    else:
        fn_return_value = CubicSolver(a, b, c, d)
    return fn_return_value

def norm(a, b, c, d):
    fn_return_value = np.sqrt(a ** 2 + b ** 2 + c ** 2 + d ** 2)
    return fn_return_value


def QuadraticSolver(a, b, c):

    # Solves a quadratic equation of the form:
    # a*x^2 + b*x + c = 0 for real roots.
    q = - 0.5 *  ( b + np.sign(b) * np.sqrt(b ** 2 - 4 * a * c) )
    x1 = q / a
    x2 = c / q
    #take the smallest positive root.
    if x1 * x2 < 0:
        fn_return_value = np.max([x1, x2])
    elif x1 > 0:
        fn_return_value = np.min([x1, x2])
    else:
        print("invalid solution for initial alpha")
    return fn_return_value

def CubicSolver(a, b, c, d):
    deg = 2.09439510239319
    tol1 = 0.00001
    tol2 = 1E-20
    z=[]
    # Solves a cubic equation of the form:
    # x^3 + b*x^2 + c*x + d = 0 for real roots.
    # Inputs:
    # b,c,d: coefficients of polynomial.
    # Outputs:
    # ROOT 3-vector containing only real roots.
    # NROOTS The number of roots found. The real roots
    # found will be in the first elements of ROOT.
    # Method: Closed form employing trigonometric and Cardan
    # methods as appropriate.
    # Note: to transform equation:
    # A*x^3 + B*x^2 + C*x + D = 0
    # into the form above, simply divide the coefficients thru by A
    # i.e. b = B/A, c = C/A and d = D/A
    b = b / a
    c = c / a
    d = d / a
    #transform the equation into the form z^3 + p*z + q = 0
    p = c - b ** 2 / 3
    q = b *  ( 2 * b ** 2 - 9 * c )  / 27 + d
    
    if np.sqrt(p ** 2 + q ** 2) < tol2:
        nr = 3
        z.append(0.0 for i in range(nr))
    else:
        g = ( p / 3 )  ** 3 +  ( q / 2 )  ** 2
        if g > 0:
            t1 = - q / 2
            t2 = np.sqrt(g)
            ratio = 1
            if q != 0:
                ratio = t2 / t1
            if np.abs(ratio) < tol1:
                nr = 3
                z.append(2 * CubicRoot(t1))
                z.append( CubicRoot(- t1))
                z.append(0.0)
            else:
                nr = 1
                z.append( CubicRoot(t1 + t2) + CubicRoot(t1 - t2))
        else:
            nr = 3
            ad3 = p / 3
            e0 = 2 * np.sqrt(- ad3)
            phi = - q /  ( 2 * np.sqrt(- ad3 ** 3) )
            phi3 = np.arccos(phi) / 3
            z.append(e0 * np.cos(phi3))
            z.append(e0 * np.cos(phi3 + deg))
            z.append(e0 * np.cos(phi3 - deg))
    root=[]
    for i in range(nr):
        root.append(z[i] - b / 3)
    fn_return_value = root[0]
    return fn_return_value

def CubicRoot(x):
    # Signed cube root function. Used by CubicSolver procedure.
    fn_return_value = np.abs(x) **  ( 1 / 3 )  * np.sign(x)
    return fn_return_value

# VB2PY (UntranslatedCode) Option Explicit


# New section

In [67]:
#@title Default title text
kk=getStrikes(0.03, 0.0, "CALL", 30)
print(kk)
volsSABRCalls=[]

volsSABRCalls=[SABR_BlackVol(0.03,strike,5.0,0.3,1.0,0.5,0.1) for strike in kk]


print(volsSABRCalls)
#for k in kk:
 # v=SABR_BlackVol(0.03,kK,5.0,0.3,1.0,0.5,0.1)
  #print(v)


[0.03, 0.039310344827586205, 0.04862068965517241, 0.057931034482758624, 0.06724137931034482, 0.07655172413793104, 0.08586206896551725, 0.09517241379310346, 0.10448275862068966, 0.11379310344827587, 0.12310344827586207, 0.13241379310344828, 0.1417241379310345, 0.1510344827586207, 0.1603448275862069, 0.1696551724137931, 0.17896551724137932, 0.18827586206896554, 0.19758620689655174, 0.20689655172413796, 0.21620689655172415, 0.22551724137931037, 0.23482758620689656, 0.24413793103448278, 0.253448275862069, 0.26275862068965516, 0.2720689655172414, 0.28137931034482766, 0.29068965517241385, 0.30000000000000004]
[0.30000000000000004, 0.307146735159562, 0.3130753804353764, 0.3181329345740638, 0.32253935777170367, 0.32644157815657354, 0.3299422213953454, 0.33311565589606085, 0.3360174361455247, 0.3386901280421481, 0.3411670513336935, 0.34347476928023785, 0.3456347954806557, 0.34766479495774566, 0.3495794488612968, 0.35139108963010396, 0.35311017592892285, 0.3547456534653755, 0.3563052330435916, 0

In [51]:
def getConvexityAdjustedSwapRate(F,D,T,swapLengthinMonths,nCashSettledSwaptions,fixedlegfreqmonths, SABRparams):
    
    #Calls
    wCalls, kCalls = calculateWeights(1,F,D,swapLengthinMonths,fixedlegfreqmonths,nCashSettledSwaptions)
    wPuts, kPuts = calculateWeights(-1,F,D,swapLengthinMonths,fixedlegfreqmonths,nCashSettledSwaptions)


    #define shifted SABR parameters
    alpha = SABRparams[0]; beta = SABRparams[1]; rho = SABRparams[2]; nu = SABRparams[3]; shift = D
    #model = SABRShifted(alpha, beta, rho, nu, D)

    #volsSABRCalls = model.black_vol(F, np.array(kCalls),T)
    #volsSABRPuts = model.black_vol(F, np.array(kPuts),T)

    #use another SABR
    volsSABRCalls=[]
    volsSABRCalls.append(SABR_BlackVol(F+D,k+D,T,alpha, beta, rho, nu) for k in kCalls)
    volsSABRPuts=[]
    volsSABRPuts.append(SABR_BlackVol(F+D,k+D,T,alpha, beta, rho, nu) for k in kPuts)

    